# Task 3: Helper notebook for loading the data and saving the predictions

In [2]:
import pickle
import gzip
import numpy as np
import os
import cv2
from copy import deepcopy

### Helper functions

In [3]:
def load_zipped_pickle(filename):
    with gzip.open(filename, 'rb') as f:
        loaded_object = pickle.load(f)
        return loaded_object

In [4]:
def save_zipped_pickle(obj, filename):
    with gzip.open(filename, 'wb') as f:
        pickle.dump(obj, f, 2)

### Load data, make predictions and save prediction in correct format

In [5]:
# load data
train_data = load_zipped_pickle("train.pkl")
test_data = load_zipped_pickle("test.pkl")
samples = load_zipped_pickle("sample.pkl")

## Get basic dataset info

In [9]:
print(f"Train data info:")
print(f"    Number of train videos: {np.array(train_data).shape[0]}")
print(f"    Min num frames in train videos: {min([data['video'].shape[2] for data in train_data])}")
print(f"    Max num frames in train videos: {max([data['video'].shape[2] for data in train_data])}")
print(f"    Average num of frames in train videos: {np.mean([data['video'].shape[2] for data in train_data])}")
# print([np.array(train_data[i]['box']).shape for i in range(len(train_data))])

print(f"Test data info:")
print(f"    Number of test videos: {np.array(test_data).shape[0]}")
print(f"    Min num frames in test videos: {min([len(data['video']) for data in test_data])}")
print(f"    Max num frames in test videos: {max([len(data['video']) for data in test_data])}")
print(f"    Average num of frames in test videos: {np.mean([len(data['video']) for data in test_data])}")

Train data info:
    Number of train videos: 65
    Min num frames in train videos: 54
    Max num frames in train videos: 334
    Average num of frames in train videos: 151.83076923076922
Test data info:
    Number of test videos: 20
    Min num frames in test videos: 582
    Max num frames in test videos: 732
    Average num of frames in test videos: 613.2


## Create videos for visualization

In [24]:
num_videos_to_save = 65

train = deepcopy(train_data)
for i in range(num_videos_to_save):
    video = train[i]['video']
    box = np.array(train[i]['box'])
    label = train[i]['label']
    label = label.astype('float32')
    print(label)
    break
#     label = label*255
#     print(list(label[train[i]['frames'][0]]))
    label = np.moveaxis(label, -1, 0)
    video = np.moveaxis(video, -1, 0)
    first_image = video[0]
    size = first_image.shape
    fps = 10
    out = cv2.VideoWriter(f'data/train_videos/{i}.mp4', cv2.VideoWriter_fourcc(*'mp4v'), fps, (size[1], size[0]), False)
    for j in range(len(video)):
        image = video[j]
#         image[box] = 255
        labelj = label[j]
        image[labelj] = 255
        box = box.astype('uint8')*255
        label = label.astype('uint8')*255
        out.write(image)
    out.release()

[[[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 ...

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]]


In [ ]:
## Create inference videos

# Load model

# Read all frames into list
# resize if needed
# feed through and get box output
# if resized, resize back.
# mask the image frame with the boxoutput and save to video



In [ ]:
# make prediction for test
predictions = []
for d in test_data:
    prediction = np.array(np.zeros_like(d['video']), dtype=np.bool)
    height = prediction.shape[0]
    width = prediction.shape[1]
    prediction[int(height/2)-50:int(height/2+50), int(width/2)-50:int(width/2+50)] = True
    
    # DATA Structure
    predictions.append({
        'name': d['name'],
        'prediction': prediction
        }
    )

In [ ]:
# save in correct format
save_zipped_pickle(predictions, 'my_predictions.pkl')